## Connector demo
compatible with EDC v0.10.1

most examples based on the edc samples: https://github.com/eclipse-edc/Samples

In [11]:
import requests
import json

from dataspace_apis import *

### Demo setup

In [12]:

IS_LOCALHOST_DEPLOYMENT = False

# for paas:
PROVIDER_URL = "https://ad4gd-provider-edc.dashboard-siba.store"
#CONSUMER_URL = "https://consumer-edc-connector.nilu.no"
CONSUMER_URL = "https://consumer-edc-connector.apps.dcw1.paas.psnc.pl"
FEDERATED_CATALOG_BASE_URL = "https://ad4gd-catalog-edc.dashboard-siba.store"
CONSUMER_BACKEND_URL = "https://consback-edc-connector.apps.dcw1.paas.psnc.pl"

#PROVIDER_URL = "https://provider-edc-connector.apps.paas-dev.psnc.pl"
#CONSUMER_URL = "https://consumer-edc-connector.apps.paas-dev.psnc.pl"
#FEDERATED_CATALOG_BASE_URL = "https://federatedcatalog-edc-connector.apps.paas-dev.psnc.pl"
#CONSUMER_BACKEND_URL = "https://consback-edc-connector.apps.paas-dev.psnc.pl"

# PROVIDER_URL = "https://provider-edc-connector.apps.dcw1.paas.psnc.pl"
# CONSUMER_URL = "https://consumer-edc-connector.apps.dcw1.paas.psnc.pl"
# FEDERATED_CATALOG_BASE_URL = "https://federatedcatalog-edc-connector.apps.dcw1.paas.psnc.pl"

## for local:
LOCALHOST = "http://localhost"
CONSUMER_CONTAINER = "http://consumer-connector"
PROVIDER_CONTAINER = "http://provider-connector"
FEDERATED_CATALOG_BASE_CONTAINER = "http://federated-catalog"
CONSUMER_BACKEND_CONTAINER = "http://consumer-backend"

if (IS_LOCALHOST_DEPLOYMENT):
    CONSUMER_URL = LOCALHOST
    PROVIDER_URL = LOCALHOST
    FEDERATED_CATALOG_BASE_URL = LOCALHOST

In [13]:
PROVIDER_API = f"{PROVIDER_URL}/api"
PROVIDER_CONTROL = f"{PROVIDER_URL}/control"
PROVIDER_MANAGEMENT = f"{PROVIDER_URL}/management"
PROVIDER_PROTOCOL = f"{PROVIDER_URL}/protocol"
PROVIDER_PUBLIC = f"{PROVIDER_URL}/public"

CONSUMER_API = f"{CONSUMER_URL}/api"
CONSUMER_CONTROL = f"{CONSUMER_URL}/control"
CONSUMER_MANAGEMENT = f"{CONSUMER_URL}/management"
CONSUMER_PROTOCOL = f"{CONSUMER_URL}/protocol"
CONSUMER_PUBLIC = f"{CONSUMER_URL}/public"

CONSUMER_BACKEND_EDR = f"{CONSUMER_BACKEND_URL}/edr-endpoint"
FEDERATED_CATALOG_URL = f"{FEDERATED_CATALOG_BASE_URL}/catalog"

if (IS_LOCALHOST_DEPLOYMENT):
    PROVIDER_API = f"{PROVIDER_URL}:19191/api"
    PROVIDER_CONTROL = f"{PROVIDER_URL}:19192/control"
    PROVIDER_MANAGEMENT = f"{PROVIDER_URL}:19193/management"
    PROVIDER_PROTOCOL = f"{PROVIDER_URL}:19194/protocol"
    PROVIDER_PUBLIC = f"{PROVIDER_URL}:19291/public"

    CONSUMER_API = f"{CONSUMER_URL}:29191/api"
    CONSUMER_CONTROL = f"{CONSUMER_URL}:29192/control"
    CONSUMER_MANAGEMENT = f"{CONSUMER_URL}:29193/management"
    CONSUMER_PROTOCOL = f"{CONSUMER_URL}:29194/protocol"
    CONSUMER_PUBLIC = f"{CONSUMER_URL}:29291/public"

    CONSUMER_BACKEND_EDR = f"{CONSUMER_BACKEND_CONTAINER}:4000/edr-endpoint"
    FEDERATED_CATALOG_URL = f"{FEDERATED_CATALOG_BASE_URL}:9181/catalog"

In [14]:
"""
The vars below are useful when working on local deployment
(or at least when no routings are specified for connectors)
Docker containers have their own localhost, which is not the host
machine's localhost.

In some requests there is a 'counterPartyAddress' which contains
localhost. This one will be solved as containers' internal localhost
but not the localhost of the host machine and connectors won't connect
to each other.

Hence we substitute the "localhost" with containers' names (if they
contain "localhost", otherwise urls remain unchanged).
If routings are specified correctly on PaaS, those lines aren't
required.

We leave those vars here anyway, just not to complicate 
any of the requests later in the demo. 
"""

provider_control_internal = PROVIDER_CONTROL.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_public_internal = PROVIDER_PUBLIC.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_protocol_internal = f"{PROVIDER_PROTOCOL}".replace(LOCALHOST, PROVIDER_CONTAINER)

default_headers = {
    "Content-Type": "application/json",
    "x-api-key": "edc",
}

### Conn Check

In [6]:
def check_health(coreApiUrl):
    print(f"{coreApiUrl}/check/health/")
    rp = requests.get(f"{coreApiUrl}/check/health/", headers=default_headers).json()
    print(rp)

check_health(PROVIDER_API)
check_health(CONSUMER_API)
print("They're Alive!")

https://ad4gd-provider-edc.dashboard-siba.store/api/check/health/
{'componentResults': [{'failure': None, 'component': 'Dataplane Self Registration', 'isHealthy': True}, {'failure': None, 'component': 'BaseRuntime', 'isHealthy': True}], 'isSystemHealthy': True}
https://consumer-edc-connector.apps.dcw1.paas.psnc.pl/api/check/health/
{'componentResults': [{'failure': None, 'component': 'Dataplane Self Registration', 'isHealthy': True}, {'failure': None, 'component': 'BaseRuntime', 'isHealthy': True}], 'isSystemHealthy': True}
They're Alive!


### Populating the provider

Not required to run, since connectors self-register data plane on startup. Just provided as an example.

In [7]:
asset_id = "test-asset"

asset = create_asset(asset_id, PROVIDER_MANAGEMENT, default_headers)

# expected 200 or 409 (already exists) because the @id is fixed in the example
asset.status_code

409

In [8]:
policy_id = "test-policy"

policy = create_policy(policy_id, PROVIDER_MANAGEMENT, default_headers, permissions=[])


"""
example with rule:

allowed_policy_region = "pl"
allowed_region_rule = {
    "action": "use", 
    "constraint": { 
        "@type": "AtomicConstraint", 
        "leftOperand": "https://w3id.org/edc/v0.0.1/ns/regionLocation", 
        "operator": "odrl:eq", 
        "rightOperand": allowed_policy_region 
    }
}
policy = create_policy(policy_id, PROVIDER_MANAGEMENT, default_headers, permissions=[allowed_region_rule])
"""

# expected 200 or 409 (already exists) because the @id is fixed in the example
print(policy.status_code)

409


Contract definition - Links an asset with a policy

In [9]:
contract_definition_id = "test-contract-definition"

contract_definition = create_contract_definition(contract_definition_id, PROVIDER_MANAGEMENT, asset_id, policy_id, default_headers)

# expected 200 or 409 (already exists) because the @id is fixed in the example
contract_definition.status_code

409

### Working on consumer side

Fetch catalog, negotiate and perform transfer

In [ ]:
import time

# 1. fetch catalog
fetched_catalog = fetch_catalog(FEDERATED_CATALOG_URL, default_headers)
print(f"1. Catalog fetch: {fetched_catalog.status_code}")

# 2. check if there is an existing negotiation
existing_negotiation = check_existing_negotiation(asset_id, CONSUMER_MANAGEMENT, default_headers)
contract_agreement_id = ""

if existing_negotiation is not None:
    # 2A. Use existing negotiation
    contract_agreement_id = existing_negotiation
    print(f"Negotiation for asset \"{asset_id}\" already exists")
else:
    offer_id = get_offer_id(fetched_catalog.json(), asset_id)
    print("OfferId:", offer_id)

    # 2B. Negotitate a contract
    negotiated_contract = negotiate_contract(
        offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [], default_headers)
    # If there's policy with rules then negotiated_contract = negotiate_contract(offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [allowed_region_rule], default_headers)

    print(f"Negotiate a contract: {negotiated_contract.status_code}")
    print(negotiated_contract.json())

    # wait a minute until the negotiation will be finalized (automatic interval process takes maximum 60 secondes to load new contracts)
    print("... Waiting for negotiation to save ...")
    time.sleep(60)

    contract_negotiation_id = negotiated_contract.json()["@id"]
    contract_agreement = get_contract_agreement_id(contract_negotiation_id, CONSUMER_MANAGEMENT, default_headers)
    print(contract_agreement.json())

    if ("contractAgreementId" in contract_agreement.json()):
        contract_agreement_id = contract_agreement.json()["contractAgreementId"]

print("[@AgreementId]:", contract_agreement_id)

# Check for existing transfer for given asset and agreement
existing_transfer = check_existing_transfer(asset_id, contract_agreement_id, CONSUMER_BACKEND_EDR, CONSUMER_MANAGEMENT, default_headers)

# 3. transfer asset
if existing_transfer is not None:
    print(f"Requested transfer already exists: {existing_transfer}")
    requested_transfer = get_transfer(existing_transfer, CONSUMER_MANAGEMENT, default_headers)
else:
    requested_transfer = request_consumer_pull_transfer(
        "provider",
        CONSUMER_MANAGEMENT,
        CONSUMER_BACKEND_EDR,
        provider_protocol_internal,
        contract_agreement_id,
        default_headers
    )

print(f"3. Transfer an asset: {requested_transfer.status_code}")
print(requested_transfer.json())

## Transfer with query params to PSNC MinIO

In [ ]:
import time

# 1. fetch catalog
fetched_catalog = fetch_catalog(FEDERATED_CATALOG_URL, default_headers)
print(f"1. Catalog fetch: {fetched_catalog.status_code}")

# 2. check if there is an existing negotiation
existing_negotiation = check_existing_negotiation(asset_id, CONSUMER_MANAGEMENT, default_headers)
print(f"Negotiation fo asset \"{asset_id}\"already exists")
contract_agreement_id = ""

if existing_negotiation is not None:
    # 2A. Use existing negotiation
    contract_agreement_id = existing_negotiation
else:
    offer_id = get_offer_id(fetched_catalog.json(), asset_id)
    print(offer_id)

    # 2. negotiate a contract
    negotiated_contract = negotiate_contract(
        offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [], default_headers)
    # If there's policy with rules then negotiated_contract = negotiate_contract(offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [allowed_region_rule], default_headers)

    print(f"2. Negotiate a contract: {negotiated_contract.status_code}")
    print(negotiated_contract.json())

    # wait a minute until the negotiation will be finalized (automatic interval process takes maximum 60 secondes to load new contracts)
    print("... Waiting for negotiation to save ...")
    time.sleep(60)

    contract_negotiation_id = negotiated_contract.json()["@id"]
    contract_agreement = get_contract_agreement_id(contract_negotiation_id, CONSUMER_MANAGEMENT, default_headers)
    print(contract_agreement.json())

    if ("contractAgreementId" in contract_agreement.json()):
        contract_agreement_id = contract_agreement.json()["contractAgreementId"]

print("[@ContractAgreementId]:", contract_agreement_id)

# 3. transfer asset (this will save the asset to PSNC MinIO)
query_params = "?url=false"
consumer_backend_url_with_query_params = CONSUMER_BACKEND_EDR + query_params

# Check for existing transfer for given asset and agreement
existing_transfer = check_existing_transfer(asset_id, contract_agreement_id, consumer_backend_url_with_query_params, CONSUMER_MANAGEMENT, default_headers)

# 3. transfer asset
if existing_transfer is not None:
    print(f"Requested transfer already exists: {existing_transfer}")
    requested_transfer = get_transfer(existing_transfer, CONSUMER_MANAGEMENT, default_headers)
else:
    requested_transfer = request_consumer_pull_transfer(
        "provider",
        CONSUMER_MANAGEMENT,
        consumer_backend_url_with_query_params,
        provider_protocol_internal,
        contract_agreement_id,
        default_headers
    )

print(f"3. Transfer an asset: {requested_transfer.status_code}")
print(requested_transfer.json())

## Local transfer

In [ ]:
import time

# 1. fetch catalog
fetched_catalog = fetch_catalog(FEDERATED_CATALOG_URL, default_headers)
print(f"1. Catalog fetch: {fetched_catalog.status_code}")

# 2. check if there is an existing negotiation
existing_negotiation = check_existing_negotiation(asset_id, CONSUMER_MANAGEMENT, default_headers)
print(f"Negotiation for asset \"{asset_id}\": {existing_negotiation}")
contract_agreement_id = ""

if existing_negotiation is not None:
    # 2A. Use existing negotiation
    contract_agreement_id = existing_negotiation
else:
    offer_id = get_offer_id(fetched_catalog.json(), asset_id)
    print(offer_id)
    # 2. negotiate a contract
    negotiated_contract = negotiate_contract(
        offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [], default_headers)
    # If there's policy with rules then negotiated_contract = negotiate_contract(offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [allowed_region_rule], default_headers)

    print(f"2. Negotiate a contract: {negotiated_contract.status_code}")
    print(negotiated_contract.json())

    # wait a minute until the negotiation will be finalized (automatic interval process takes maximum 60 secondes to load new contracts)
    print("... Waiting for negotiation to save ...")
    time.sleep(60)

    contract_negotiation_id = negotiated_contract.json()["@id"]
    contract_agreement = get_contract_agreement_id(contract_negotiation_id, CONSUMER_MANAGEMENT, default_headers)
    print(contract_agreement.json())

    if ("contractAgreementId" in contract_agreement.json()):
        contract_agreement_id = contract_agreement.json()["contractAgreementId"]

print("[@ContractAgreementId]:", contract_agreement_id)

# 3. transfer asset (this will save the asset to PSNC MinIO)
query_params = "?url=false"
consumer_backend_url_with_query_params = CONSUMER_BACKEND_EDR + query_params

# Check for existing transfer for given asset and agreement
existing_transfer = check_existing_transfer(asset_id, contract_agreement_id, consumer_backend_url_with_query_params, CONSUMER_MANAGEMENT, default_headers)

# 3. transfer asset
if existing_transfer is not None:
    print(f"Requested transfer already exists: {existing_transfer}")
    requested_transfer = get_transfer(existing_transfer, CONSUMER_MANAGEMENT, default_headers)
else:
    requested_transfer = request_consumer_pull_transfer(
        "provider",
        CONSUMER_MANAGEMENT,
        consumer_backend_url_with_query_params,
        provider_protocol_internal,
        contract_agreement_id,
        default_headers
    )

print("... Waiting for transfer to save ...")
time.sleep(10)

print(f"3. Transfer an asset: {requested_transfer.status_code}")
print(requested_transfer.json())

print("4. Get credentials")
print(f"{CONSUMER_MANAGEMENT}/v3/edrs/{requested_transfer.json()["@id"]}/dataaddress")
transfer_credentials = get_transfer_data_credentials(CONSUMER_MANAGEMENT, requested_transfer.json()["@id"], default_headers)
print(transfer_credentials)

proxy_path = "formats/html"
query_params = "url=true"
public = f"{transfer_credentials['endpoint']}/{proxy_path}?{query_params}"

if IS_LOCALHOST_DEPLOYMENT:
    public = public.replace(PROVIDER_CONTAINER, LOCALHOST)
    
data = get_data_locally(public, transfer_credentials['authorization'])
if (data.text):
    print(data.text)
else:
    print(data.content)
    

1. Catalog fetch: 200
Negotiation for asset "test-asset": a01dda71-4c75-44fb-ab26-7efbe974cab5
[@ContractAgreementId]: a01dda71-4c75-44fb-ab26-7efbe974cab5
Requested transfer already exists: 014fd1cf-5bc7-4f30-a487-364744f2d067
... Waiting for transfer to save ...
3. Transfer an asset: 200
{'@id': '014fd1cf-5bc7-4f30-a487-364744f2d067', '@type': 'TransferProcess', 'state': 'STARTED', 'stateTimestamp': 1750096798399, 'type': 'CONSUMER', 'callbackAddresses': {'@type': 'CallbackAddress', 'transactional': False, 'uri': 'https://consback-edc-connector.apps.dcw1.paas.psnc.pl/edr-endpoint?url=false', 'events': 'transfer.process.started'}, 'correlationId': '7248bd53-ca74-4465-8e83-29c921cfd4b0', 'assetId': 'test-asset', 'contractId': 'a01dda71-4c75-44fb-ab26-7efbe974cab5', 'transferType': 'HttpData-PULL', 'dataDestination': {'@type': 'DataAddress', 'type': 'HttpProxy'}, '@context': {'@vocab': 'https://w3id.org/edc/v0.0.1/ns/', 'edc': 'https://w3id.org/edc/v0.0.1/ns/', 'odrl': 'http://www.w3.or

TypeError: list indices must be integers or slices, not str